# GridSearch para regresión (ventas)

**Candidatos**
- BaggingRegressor
- GradientBoostingRegressor
- RandomForestRegressor

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd
import re
import numpy as np
np.seterr(divide='ignore', invalid='ignore');  # warnings inutiles apagados!

In [2]:
def custom_features(dataframe_in):
    df = dataframe_in.copy(deep=True)

    df['month'] = pd.to_datetime(df['release_date']).dt.month
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.to_julian_date())

    df['revenue'] = pd.Series([0 for _ in range(len(dataframe_in))])

    df.loc[df.publisher.str.match('.*microsoft.*', flags=re.IGNORECASE).values, 'revenue'] = 10.260
    df.loc[df.publisher.str.match('.*netease.*', flags=re.IGNORECASE).values, 'revenue'] = 6.668
    df.loc[df.publisher.str.match('.*activision.*', flags=re.IGNORECASE).values, 'revenue'] = 6.388
    df.loc[df.publisher.str.match('.*electronic.*', flags=re.IGNORECASE).values, 'revenue'] = 5.537
    df.loc[df.publisher.str.match('.*bandai.*', flags=re.IGNORECASE).values, 'revenue'] = 3.018
    df.loc[df.publisher.str.match('.*square.*', flags=re.IGNORECASE).values, 'revenue'] = 2.386
    df.loc[df.publisher.str.match('.*nexon.*', flags=re.IGNORECASE).values, 'revenue'] = 2.286
    df.loc[df.publisher.str.match('.*ubisoft.*', flags=re.IGNORECASE).values, 'revenue'] = 1.446
    df.loc[df.publisher.str.match('.*konami.*', flags=re.IGNORECASE).values, 'revenue'] = 1.303
    df.loc[df.publisher.str.match('.*SEGA.*').values, 'revenue'] = 1.153
    df.loc[df.publisher.str.match('.*capcom.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7673
    df.loc[df.publisher.str.match('.*warner.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7324

    return df

In [3]:
from preprocessing import Nothing, CategoriesTokenizer, custom_features


boc_some_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 0.05  # hiperparametro a optimizar
    # valores para GridSearch : [5%, 10%, 15%] ???
    )


boc_many_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 1  # hiperparametro a optimizar
    # valores para GridSearch : [5, 10, 15] ???
    )


preprocessing = ColumnTransformer(
    transformers=[
        ('BoC-plat',boc_some_values,'platforms'),
        ('BoC-cat',boc_some_values,'categories'),
        ('BoC-genres',boc_some_values,'genres'),
        ('BoC-tags',boc_some_values,'tags'),

        ('BoC-dev',boc_many_values,'developer'),
        ('BoC-pub',boc_many_values,'publisher'),

        ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'),['month']),
        ('MinMaxScaler',MinMaxScaler(),['required_age','price','release_date']),
        ('BoxCox',PowerTransformer(method='yeo-johnson'),['achievements','average_playtime','revenue']),
        ('unchanged',Nothing(),['english'])
])

In [4]:
df_train = pd.read_pickle('train.pickle')
df_train = custom_features(df_train)

#########################
df_train = df_train[:100]
#########################

X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['estimated_sells'], test_size=0.3, random_state=0)

In [9]:
df_train['revenue']

KeyError: 'revenue'

## Regresor Bagging

In [5]:
from sklearn.ensemble import BaggingRegressor

In [6]:
pipeline = Pipeline(steps = [
    ('procesamiento',preprocessing),
    ("selector", SelectPercentile(f_classif, percentile=95)),
    ("regressor",BaggingRegressor(random_state=0)),
])

grilla_bagging = {
    "regressor": [BaggingRegressor(random_state=0)],
    "selector__percentile" : [20, 40, 80, 95, 100],
    "selector__score_func" : [f_classif],
    "regressor__n_estimators" : [5, 10, 15, 20],
    # 'regressor__base_estimator__max_depth' : [1, 4, 8, None],
    'regressor__max_samples' : [0.05, 0.2, 0.5, 1.0]
}

In [7]:
gs = HalvingGridSearchCV(
    pipeline,
    grilla_bagging,
    scoring = 'r2',
    # n_jobs=-1,
    verbose = 10
)

gs.fit(X_train, y_train)

n_iterations: 2
n_required_iterations: 4
n_possible_iterations: 2
min_resources_: 10
max_resources_: 70
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 80
n_resources: 10
Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5; 1/80] START regressor=BaggingRegressor(random_state=0), regressor__max_samples=0.05, regressor__n_estimators=5, selector__percentile=20, selector__score_func=<function f_classif at 0x7fea329972e0>
[CV 1/5; 1/80] END regressor=BaggingRegressor(random_state=0), regressor__max_samples=0.05, regressor__n_estimators=5, selector__percentile=20, selector__score_func=<function f_classif at 0x7fea329972e0>;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/5; 1/80] START regressor=BaggingRegressor(random_state=0), regressor__max_samples=0.05, regressor__n_estimators=5, selector__percentile=20, selector__score_func=<function f_classif at 0x7fea329972e0>
[CV 2/5; 1/80] END regressor=BaggingRegressor(random_state=0), regressor__max

ValueError: 
All the 400 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'revenue'

The above exception was the direct cause of the 

El mejor modelo encontrado es el siguiente

In [8]:
gs.best_params_ 

AttributeError: 'HalvingGridSearchCV' object has no attribute 'best_params_'

Y sus métricas son:

In [ ]:
print("Resultados clasificador ganador de GridSearch Bagging Regressor")

y_pred = gs.predict(X_eval)
print("Error cuadrático medio = {}".format(mean_squared_error(y_eval,y_pred)))
print("Score R2 = {}".format(r2_score(y_eval,y_pred)))

## Regresor GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

## Regresor RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor